Next steps:
 * How does numpydoc format deprecation warnings?
 * ``UserWarning`` to ``DeprecationWarning``
 * experimental ``until`` to ``as-of``
 * deprecated should also take ``as-of`` in addition to ``until``
 * force naming of kwargs for readability
 * update ``deprecated``'s ``wrapped_f.__name__``, etc (see [link from Evan](http://stackoverflow.com/questions/147816/preserving-signatures-of-decorated-functions))

In [1]:
from warnings import warn
from textwrap import wrap

class state_decorator(object):
    
    _line_header = '\n        '
    
    def _update_doc_string(self, func, state_desc):
        doc_lines = func.__doc__.split('\n')
        state_desc_lines = wrap('State: ' + state_desc, 79 - len(self._line_header))
        doc_lines.insert(1, self._line_header + self._line_header.join(state_desc_lines))
        return '\n'.join(doc_lines)
        
class stable(state_decorator):
    
    def __init__(self, as_of):
        self.as_of = as_of
    
    def __call__(self, func):
        state_desc = 'Stable as of %s.' % self.as_of
        func.__doc__ = self._update_doc_string(func, state_desc)
        return func

class experimental(state_decorator):
    
    def __init__(self, until):
        self.until = until
    
    def __call__(self, func):
        state_desc = 'Experimental until %s.' % self.until
        func.__doc__ = self._update_doc_string(func, state_desc)
        return func

class deprecated(state_decorator):
    
    def __init__(self, until, reason):
        self.until = until
        self.reason = reason
    
    def __call__(self, func, *args, **kwargs):
        state_desc = 'Deprecated until %s. %s' % (self.until, self.reason)
        func.__doc__ = self._update_doc_string(func, state_desc)
        def wrapped_f(*args, **kwargs):
            warn('%s is deprecated and will be removed in scikit-bio version %s. %s' %
                 (func.__name__, self.until, self.reason)) 
            return func(*args, **kwargs)
        wrapped_f.__doc__ = func.__doc__
        return wrapped_f

class A(object):
    
    @stable('0.3.1')
    def return_x(self, x):
        """This method returns its input.
        
        Probably not the most useful function ever.
        
        Parameters
        ----------
        x : the value to be returned
        
        Returns
        -------
        whatever was passed in
        """
        return x

    @experimental('0.3.4')
    def return_y(self, y):
        """This method returns its input.
        
        Probably not the most useful function ever.
        
        Parameters
        ----------
        y : the value to be returned
        
        Returns
        -------
        whatever was passed in
        """
        return y

    @deprecated('0.3.4', 'You should now be using B.return_z1.')
    def return_z1(self, z1):
        """This method returns its input.
        
        Probably not the most useful function ever.
        
        Parameters
        ----------
        z1 : the value to be returned
        
        Returns
        -------
        whatever was passed in
        """
        return z1

    @deprecated('0.3.4', 'This method was shown to be error-prone by [Foo et al. 2015](www.pubmed.gov/12345)')
    def return_z2(self, z2):
        """This method returns its input.
        
        Probably not the most useful function ever.
        
        Parameters
        ----------
        z2 : the value to be returned
        
        Returns
        -------
        whatever was passed in
        """
        return z2

In [2]:
help(A().return_x)

Help on method return_x in module __main__:

return_x(self, x) method of __main__.A instance
    This method returns its input.
    
    State: Stable as of 0.3.1.
    
    Probably not the most useful function ever.
    
    Parameters
    ----------
    x : the value to be returned
    
    Returns
    -------
    whatever was passed in



In [3]:
help(A().return_y)

Help on method return_y in module __main__:

return_y(self, y) method of __main__.A instance
    This method returns its input.
    
    State: Experimental until 0.3.4.
    
    Probably not the most useful function ever.
    
    Parameters
    ----------
    y : the value to be returned
    
    Returns
    -------
    whatever was passed in



In [4]:
print A().return_z1(42)

42


/Users/caporaso/.virtualenvs/marisa-trie/lib/python2.7/site-packages/IPython/kernel/__main__.py:45: UserWarning: return_z1 is deprecated and will be removed in scikit-bio version 0.3.4. You should now be using B.return_z1.


In [5]:
print A().return_z2(42)

42


/Users/caporaso/.virtualenvs/marisa-trie/lib/python2.7/site-packages/IPython/kernel/__main__.py:45: UserWarning: return_z2 is deprecated and will be removed in scikit-bio version 0.3.4. This method was shown to be error-prone by [Foo et al. 2015](www.pubmed.gov/12345)


In [6]:
help(A().return_z1)

Help on method wrapped_f in module __main__:

wrapped_f(*args, **kwargs) method of __main__.A instance
    This method returns its input.
    
    State: Deprecated until 0.3.4. You should now be using B.return_z1.
    
    Probably not the most useful function ever.
    
    Parameters
    ----------
    z1 : the value to be returned
    
    Returns
    -------
    whatever was passed in



In [7]:
help(A().return_z2)

Help on method wrapped_f in module __main__:

wrapped_f(*args, **kwargs) method of __main__.A instance
    This method returns its input.
    
    State: Deprecated until 0.3.4. This method was shown to be error-prone
    by [Foo et al. 2015](www.pubmed.gov/12345)
    
    Probably not the most useful function ever.
    
    Parameters
    ----------
    z2 : the value to be returned
    
    Returns
    -------
    whatever was passed in

